In [30]:
using Pkg
Pkg.activate("MISAEM")

using MISAEM

  Activating project at `c:\Users\Chris\Documents\INRIA\R_scripts\misaem_julia\MISAEM`


In [31]:
using MISAEM
using Random
using Distributions
using LinearAlgebra
using Statistics

# Set a random seed for reproducibility
Random.seed!(123)

# 2. Generate some example data
# n: number of samples, p: number of features
n = 5000
p = 5

# Create a design matrix X with some missing values (NaN)
X_complete = rand(Normal(), n, p)
X_missing = copy(X_complete)
X_missing[rand(Bool, n, p) .< 0.15] .= NaN # Introduce 15% missing values

# Create a response vector y (binary)
true_coeffs = randn(p)
linear_predictor = X_complete[:, 1:p] * true_coeffs
probabilities = 1.0 ./ (1.0 .+ exp.(-linear_predictor))
y = rand.(Bernoulli.(probabilities));

println(probabilities[1:10])
println(y[1:10])
println(mean(y))

[0.8142665605438179, 0.9877458230632161, 0.02428674270954094, 0.9609970544880675, 0.03080470492392332, 0.995934089641997, 0.9993806948719941, 0.8159392823782796, 0.4140418172841337, 0.8878416396716566]
Bool[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
0.4988


In [32]:


# 3. Instantiate the MISAEM model
# You can adjust hyperparameters here if needed
model = MISAEM.SAEMLogisticRegression(
    maxruns = 1000,
    random_state = 42,
    var_cal=false,
    ll_obs_cal=false
)


SAEMLogisticRegression(1000, 1.0e-7, 2, 1.0, 50, false, false, nothing, 42, Float64[], 0.0, Float64[], Float64[], Matrix{Float64}(undef, 0, 0), 0.0, false, 0)

In [33]:

# 4. Fit the model to the data
println("Fitting the model...")
MISAEM.fit!(model, X_missing, y)


Fitting the model...


┌ Warning: 166 rows with all NaN values in X have been removed.
│   DataRemovalWarning = MISAEM.DataRemovalWarning
└ @ MISAEM C:\Users\Chris\Documents\INRIA\R_scripts\misaem_julia\MISAEM\src\MISAEM.jl:121
SAEM Iterations:  26%|█████████                         |  ETA: 0:00:08


...converged after 264 iterations.


SAEMLogisticRegression(1000, 1.0e-7, 2, 1.0, 50, false, false, nothing, 42, [-0.07823509429975847, -1.1051687816165754, -1.7694085982690868, -2.2711008741860983, -0.9619462570617524, -1.6928029923530103], -0.07823509429975847, Float64[], [0.00014733567125579824, -0.023711784351258765, -0.024161966214708668, 0.02810790040674866, -0.0018869683802455839], [0.9717678730923467 -0.0278694842379803 … -0.05414681106964388 0.024895347360842493; -0.0278694842379803 1.0046914941587706 … -0.014020356568390058 0.0456590617212641; … ; -0.05414681106964388 -0.014020356568390058 … 1.0051710729410817 0.01673778198773627; 0.024895347360842493 0.0456590617212641 … 0.01673778198773627 1.0454909831542754], 0.0, true, 264)

In [34]:

# 5. Display the results
println("\nFitting complete.")
println("Converged: ", model.converged)
println("Number of iterations: ", model.n_iterations)
println("Estimated Coefficients (including intercept):")
println(model.coef)
println("Observed Log-Likelihood: ", model.ll_obs)

# 6. Make predictions on the training data
println("\nMaking predictions...")
predictions = MISAEM.predict(model, X_missing)
probabilities = MISAEM.predict_proba(model, X_missing, method=:map)

println("Prediction for the first 5 samples: ", predictions[1:5])
println("Probabilities for the first 5 samples: ", probabilities[1:5, :])


Fitting complete.
Converged: true
Number of iterations: 264
Estimated Coefficients (including intercept):
[-0.07823509429975847, -1.1051687816165754, -1.7694085982690868, -2.2711008741860983, -0.9619462570617524, -1.6928029923530103]
Observed Log-Likelihood: 0.0

Making predictions...


┌ Warning: 166 rows with all NaN values in X have been removed.
│   DataRemovalWarning = MISAEM.DataRemovalWarning
└ @ MISAEM C:\Users\Chris\Documents\INRIA\R_scripts\misaem_julia\MISAEM\src\MISAEM.jl:121


DimensionMismatch: DimensionMismatch: a has size (1, 3), b has size (3,), mismatch at dim 1

In [36]:
true_coeffs

5-element Vector{Float64}:
 -1.066850058222926
 -1.7049308411679553
 -2.2137526751489816
 -1.0250537232983983
 -1.6547101386351437